In [32]:
import numpy as np
import scipy as sp
import scipy.stats
import matplotlib.pyplot as plt
import networkx as nx
from collections import namedtuple

# Завдання 5
Оцінити кількість розфарбовок графа

In [41]:
def get_neighbours(A, from_node):
    return np.arange(A.shape[0])[A[from_node] != 0]

def get_available_colors(k, adjacent_colors):
    all_colors = set(range(k))
    unavailable = set(adjacent_colors)
    return list(all_colors.difference(unavailable))

def color(A, k, from_node, rng: np.random.Generator):
    # BFS walk.
    n = A.shape[0]
    colors = np.ones(n) * -1
    todo = [ from_node ]
    while len(todo) > 0:
        current = todo.pop()
        neighbours = get_neighbours(A, current)
        a_colors = get_available_colors(k, colors[neighbours])
        if len(a_colors) == 0 and colors[current] == -1:
            raise ValueError("Cannot proceed: no available colors.")
        colors[current] = rng.choice(a_colors)
        for x in neighbours:
            if colors[x] == -1:
                todo.append(x)
    return colors

def random_graph(n, p=0.3):
    A = (np.random.rand(n,n) < p)
    A = (A | A.T) & ((1-np.eye(n)) > 0)
    A = A.astype(np.uint8)
    return A

def draw(A,colors):
    G=nx.to_networkx_graph(A)
    nx.draw_networkx(G, node_color=colors)
        

# A - adjacency matrix
# k - кількість кольорів у розфарбовці
Task5Params = namedtuple('Task5Params', 'A,k')
Task5Result = namedtuple('Task5Result', 'nsteps, history')
def Task5(params: Task5Params, stat_offset=2000, n_steps=2000, seed=42, debug=False):
    rng = np.random.default_rng(seed)
    A,k = params
    n = A.shape[0]
    if A.shape[0] != A.shape[1]: raise ValueError("Not square adjacency matrix A")
    if (A != A.T).any(): raise ValueError("Not symmetric adjacency matrix A")

    # todo.
        
    return Task5Result(n_steps, history)
